# Scheduling Surgeries
## Interger Programming Example
This problem schedules patients into their preferred timeslots.

In [1]:
from gurobipy import *
import numpy as np

### Setting up indices, parameters.
Let's work with an example with 20 patients, and 6 time slots.

In [2]:
n = 20 # 20 patients
T = 6 # 4 markets
patients = range(n)
periods = range(T)

Set up patient preference:
#### IMPORTANT: This patient preference table is given to us, and the values are *input parameters*. Don't confuse them with the *binary decision variables* just because they both happen to contain binary values.

In [3]:
# Patient preference. Each row corresponds to a patient, each column correponds to a time period.
alpha = np.array([[1,1,0,0,0,1],
                  [0,1,1,0,0,0],
                  [0,1,0,0,0,1],
                  [0,0,0,0,1,1],
                  [0,1,0,0,1,1],
                  [0,1,0,0,0,1],
                  [1,0,0,0,1,0],
                  [0,0,1,1,0,0],
                  [0,0,1,0,0,0],
                  [0,0,0,1,0,0],
                  [0,1,0,0,1,0],
                  [1,1,1,1,1,1],
                  [0,0,0,0,0,0],
                  [1,1,0,0,0,1],
                  [0,0,0,0,0,0],
                  [0,1,1,0,0,1],
                  [0,0,0,1,1,1],
                  [0,1,1,1,0,0],
                  [0,0,1,0,1,0],
                  [0,0,0,0,1,0]])

Import doctor availability input parameters below.

*Assumption*: the *total* number of rooms is not a bottleneck in any time period. Therefore, we do not explicitly include any constraints related to the number of rooms available.



In [4]:
C = np.array([5,5,5,3,3,3])

### Setting up the model.

In [5]:
m = Model("scheduling")

Academic license - for non-commercial use only - expires 2022-08-29
Using license file /Users/phoebe/gurobi.lic


Set up decision variables:

To define binary variables, add "vtype=GRB.BINARY" in the variable definition. This way Gurobi knows that each x[i,t] should be constrained to take value 0 or 1. 

In [6]:
# x[i,t] == 1 if we put patient i in time period t; x[i,t] == 0 if we do not put patient i in time period t.
# This defines 6 x 20 = 120 decision variables simultaneously, one for each possible (patient, time period) match

x = m.addVars(patients, periods, vtype=GRB.BINARY)

Set objective function.
Here because the objective function is very long, we initialize an LinExpr object to hold it first, and pass the LinExpr obj to setObjective function later.

In [7]:
# The objective is to maximize the total number of patients matched to a preferred time

m.setObjective(sum(sum(alpha[i,t] * x[i,t] for i in patients) for t in periods), GRB.MAXIMIZE)

Now set all the constraints:

In [8]:
# For every patient, we want to schedule this patient
for i in patients:
    m.addConstr(sum(x[i,t] for t in periods) == 1)
    
# For every time period, we should not schedule more patients in this period than the number of available surgeons
for t in periods:
    m.addConstr(sum(x[i,t] for i in patients) <= C[t])

### Solving the model.
 
Integer program models can be much harder to solve than linear programs. It may very well be that a seemingly simple model can take hours or days to solve. So it is always a good computational practice to explicit set the solver run time.

In [9]:
m.Params.TimeLimit = 60 # seconds

Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [11]:
# Solve
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 26 rows, 120 columns and 240 nonzeros
Model fingerprint: 0xc13497e5
Variable types: 0 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolved: 26 rows, 120 columns, 240 nonzeros

Continuing optimization...


Explored 0 nodes (38 simplex iterations) in 0.07 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 18 9 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.800000000000e+01, best bound 1.800000000000e+01, gap 0.0000%


In this case, it only takes a fraction of a second to solve, so our time limit was not activated.

Print the solution, i.e., objective value and the optimal variable values:

In [12]:
print("Retrieve Optimal Solution.")

# Print objective value
print("Objective value =", m.objVal)

print("***")

# Print the time period where the patient is scheduled into
for i in patients:
    checkSum = 0
    scheduled_period = -1
    for t in periods:
        checkSum += x[i,t].x
        if x[i,t].x == 1.0:
            scheduled_period = t
    if checkSum > 1:
        print("Patient", i, "scheduled more than once, error.")
    print("Patient", i, "is scheduled to time period", scheduled_period)
    

print("***")

# Print the number of patients for each period
for t in periods:
    num_patients = 0
    for i in patients:
        num_patients += x[i,t].x
    print("In time period", t, "we scheduled", num_patients, "patients (", C[t], "surgeons available this period).")

Retrieve Optimal Solution.
Objective value = 18.0
***
Patient 0 is scheduled to time period 1
Patient 1 is scheduled to time period 1
Patient 2 is scheduled to time period 1
Patient 3 is scheduled to time period 4
Patient 4 is scheduled to time period 5
Patient 5 is scheduled to time period 5
Patient 6 is scheduled to time period 0
Patient 7 is scheduled to time period 2
Patient 8 is scheduled to time period 2
Patient 9 is scheduled to time period 3
Patient 10 is scheduled to time period 1
Patient 11 is scheduled to time period 5
Patient 12 is scheduled to time period 2
Patient 13 is scheduled to time period 0
Patient 14 is scheduled to time period 0
Patient 15 is scheduled to time period 2
Patient 16 is scheduled to time period 3
Patient 17 is scheduled to time period 2
Patient 18 is scheduled to time period 4
Patient 19 is scheduled to time period 4
***
In time period 0 we scheduled 3.0 patients ( 5 surgeons available this period).
In time period 1 we scheduled 4.0 patients ( 5 surge

### Discussion 1

* What does the objective value mean in our case?
* What happens if we change the surgeon capacity vector to [3,3,3,3,3,3]? i.e., 3 surgeons are available at each time period.
    * Without using code, what do you think would happen?
    * With code, what do you observe?

### Extension 1: Urgent care

Some patients require urgent care, meaning they need to be scheduled to the first two periods (period 0 or 1).
Assume this urgent care **parameter** set is given as a vector u. 

In [13]:
u = np.array([1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0])
# patients 0, 2, 5, 10, 14 need urgent care


In [14]:
for i in patients:
    m.addConstr(sum(u[i] * x[i,t] for t in periods[2:]) == 0)

Note that we are not implementing the optimization model m from scratch.
We are simply adding the additional, new constraints to the model.
This is okay even though we have already "solved" m.

In [ ]:
# Resolve the model with the additional set of constraints.
m.optimize()

Now let's check the solution and make sure patients needing urgent care are indeed scheduled to periods 0 or 1.

In [ ]:
for i in patients:
    if u[i] > 0:
        scheduled_period = -1
        for t in periods:
            if x[i,t].x == 1.0:
                scheduled_period = t
        print("Patient", i, "requires urgent care, and is scheduled to period ", scheduled_period)

In [ ]:
print("***")

# Print the time period that the patient is scheduled into
for i in patients:
    checkSum = 0
    scheduled_period = -1
    for t in periods:
        checkSum += x[i,t].x
        if x[i,t].x == 1.0:
            scheduled_period = t
    if checkSum > 1:
        print("Patient", i, "scheduled more than once, error.")
    print("Patient", i, "is scheduled to time period", scheduled_period)
    

print("***")

# Print the number of patients for each period
for t in periods:
    num_patients = 0
    for i in patients:
        num_patients += x[i,t].x
    print("In time period", t, "we scheduled", num_patients, "patients (", C[t], "surgeons available this period).")

### Extension 2: Specialized surgery
Some patients require orthopedics surgery, and only two operating rooms have the required equipment to perform orthopedics surgery.

*Assumption*: the *total* number of rooms is not a bottleneck in any time period. Therefore, we do not explicitly include any constraints related to the number of rooms available.

Suppose we have **input parameters** o_i, telling us which patients need orthopedics surgery.

In [ ]:
o = np.array([1,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,0]) # o[i] is 1 if patient i requires orthopedics surgery

Add the constraint: for each time period t, we can not schedule more than 2 patients that require orthopedics surgery

In [ ]:
for t in periods:
    m.addConstr(sum(o[i] * x[i,t] for i in patients) <= 2)

In [ ]:
m.optimize()

In [ ]:
print("***")

# Print the time period where the patient is scheduled into
for i in patients:
    checkSum = 0
    scheduled_period = -1
    for t in periods:
        checkSum += x[i,t].x
        if x[i,t].x == 1.0:
            scheduled_period = t
    if checkSum > 1:
        print("Patient", i, "scheduled more than once, error.")
    print("Patient", i, "is scheduled to time period", scheduled_period)
    

print("***")

# Print the number of patients for each period
for t in periods:
    num_patients = 0
    for i in patients:
        num_patients += x[i,t].x
    print("In time period", t, "we scheduled", num_patients, "patients (", C[t], "surgeons available this period).")
    
print("***")

for i in patients:
    if u[i] > 0:
        scheduled_period = -1
        for t in periods:
            if x[i,t].x == 1.0:
                scheduled_period = t
        print("Patient", i, "requires urgent care, and is scheduled to period ", scheduled_period)
        
print("***")

for t in periods:
    num_orthopedics = 0
    for i in patients:
        if o[i] == 1 and x[i,t].x == 1:
            num_orthopedics += 1
    print("Period", t, "has", num_orthopedics, "orthopedics patients")

### Discussion 2
* Did the objective value change in the previous three solutions? What does that imply?

### Extension 3: Patient 3 can not do surgery after patient 4
Let's say i=2 is patient 3, and i=3 is patient 4

In [ ]:
m.addConstr(sum(x[2,t] * t for t in periods) <= sum(x[3,t] * t for t in periods))

In [ ]:
m.optimize()

In [ ]:
print("Patient 3 is scheduled in period", sum(x[2,t].x * t for t in periods))
print("Patient 4 is scheduled in period", sum(x[3,t].x * t for t in periods))

### Visualizing the final schedule

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

final_schedule = np.zeros([n, T], dtype=bool)

for i in patients:
    for t in periods:
        final_schedule[i,t] = x[i,t].x

ax = sns.heatmap(final_schedule, linewidths=5, cmap="YlGnBu", square=False, cbar=False)

ax.set(xlabel="Time Slot", ylabel="Patient #")

plt.show()